In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

In [ ]:
using System.IO;

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'D:\local\OscillatingDroplet3D'.


In [ ]:
//wmg.Sessions

In [ ]:
int numSess = wmg.Sessions.Count();
numSess

86

In [ ]:
//add database manually:
OpenOrCreateDatabase(@"\\dc1\userspace\smuda\Databases\OscillatingDroplet3D");
//OpenOrCreateDatabase(@"\\130.83.248.207\ValidationTests\OscillatingDroplet3D");

Opening existing database '\\dc1\userspace\smuda\Databases\OscillatingDroplet3D'.


In [ ]:
var studySess = wmg.Sessions.Skip(numSess).Take(16);
studySess

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005*	04/08/2022 11:37:03	92930af7...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01*	04/08/2022 11:36:37	15341e01...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04*	04/08/2022 11:34:56	8486c026...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04*	04/08/2022 11:34:00	e3a04ed7...
#4: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01*	04/08/2022 11:33:27	59bb73dc...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02*	04/08/2022 11:33:07	2e275baf...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04*	04/08/2022 11:32:42	8ef825a4...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_thirdOrderInit*	04/08/2022 11:32:17	adf2022d...
#8: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_thirdOrderInit*	04/08/2022 11:31:55	106a329f...
#9: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_thirdOrderInit*	04/08/2022 11:31:22	5f8

In [ ]:
// var origin = databases.Pick(5);
// origin

{ Session Count = 38; Grid Count = 183; Path = \\dc1\userspace\smuda\Databases\OscillatingDroplet3D }

In [ ]:
var target = databases.Pick(5);
target

{ Session Count = 43; Grid Count = 80; Path = D:\local\OscillatingDroplet3D }

In [ ]:
//origin.Sessions.Pick(8).Copy(target);

In [ ]:
string[] caseS = new string[] { "m2_Oh01_eta04", "m2_Oh01_eta02", "m2_Oh01_eta01", "m3_Oh01_eta04", "m4_Oh01_eta04", "m4_Oh01_eta01", "m4_Oh056_eta005"};
string[] gridS = new string[] { "J686", "J432" };
bool[] ShouldAnaInit = new bool[] { true, false };
bool[] useNewton = new bool[] { false };

In [ ]:
Plot2Ddata[,] PlotTableDM = new Plot2Ddata[caseS.Length,1];
for(int iCol = 0; iCol < 1; iCol++) {
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
for(int iGrd = 0; iGrd < gridS.Length; iGrd++) {   
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {

    string _case = caseS[iRow];
    string _grd = gridS[iGrd];

    // if(_case == "case2" || _case == "case3")
    //     continue; 

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_grd)
                                      && (sess.Name.Contains("_thirdOrderInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));


    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    if(SIs.IsNullOrEmpty()) 
        continue;

    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("DropletMetrics.txt", '\t');
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        // var dummyPlot = new Plot2Ddata();
        // dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        // PlotTableDM[iRow,iCol] = dummyPlot;
        continue;
    }
    
    var plot = new Plot2Ddata();
    var time = tab["time"];

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    if (useNewton[iNew]) {
        fmt.LineColor = LineColors.Red;
        //fmt.DashType = DashTypes.Dashed;
    } else {  
        fmt.LineColor = LineColors.Blue;
        //fmt.DashType = DashTypes.Solid;
    }
    if (!ShouldAnaInit[iAna]) {
        fmt.DashType = DashTypes.Solid;
    } else {
        fmt.DashType = DashTypes.Dashed;
    }
    
    // if (_init == "_initCase0")
    //     fmt.LineColor = LineColors.Green;
    // if (_init == "_initCase1")
    //     fmt.LineColor = LineColors.Magenta;
    // if (_init == "_initCase2")
    //     fmt.LineColor = LineColors.Orange;
 
    string name = "BoSSS-"+_grd;
    // if(iAMR > 0)
    //     name = name + "-amr";
    if(ShouldAnaInit[iAna])
        name = name + "-3OrdInit"; 
    if(useNewton[iNew])
        name = name + "-Newton";

    double[] L = tab["L"].ToArray(); 
    double[] Wx = tab["Wx"].ToArray(); 
    double[] Wy = tab["Wy"].ToArray();
    double[] arLW = new double[L.Length];
    for (int i = 0; i < L.Length; i++) {
        arLW[i] = 0.5 * ((L[i] / (2.0*Wx[i])) + (L[i] / (2.0*Wy[i])));
    }
    
    // List<double> arLW_lst = new List<double>();
    // List<double> time_lst = new List<double>();
    // double dataThreshold = 0.01;
    // for(int i = 0; i < L.Length; i++) {
    //     if(arLW[i] > dataThreshold) {
    //         arLW_lst.Add(arLW[i]);
    //         time_lst.Add(time[i]);
    //     }
    // }
    // arLW = arLW_lst.ToArray();
    // time = time_lst.ToArray();

    plot.AddDataGroup(name, time, arLW, fmt);
   
    plot.ShowLegend = true;  

    if(PlotTableDM[iRow,iCol]==null) {
        PlotTableDM[iRow,iCol] = plot;
    } else {
        PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    }
        
    // if(iAMR == 0) {
    //     PlotTableDM[iRow,iCol] = plot;
    // } else {
    //     PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    // }
    
}

if(PlotTableDM[iRow,iCol] != null) {
    PlotTableDM[iRow,iCol].ShowLegend = true; 
    //PlotTableDM[iRow,iCol].LegendFont = 10;
    if(iCol == 0) {
       PlotTableDM[iRow,iCol].Title = caseS[iRow];
       PlotTableDM[iRow,iCol].Xlabel = "time t";
       PlotTableDM[iRow,iCol].Ylabel = "aspect ratio L/W";
    }
}
}
}
}
}

In [ ]:
var pltTab = PlotTableDM[3,0];
pltTab.Title

m3_Oh01_eta04

In [ ]:
pltTab.XrangeMin = 0;
pltTab.XrangeMax = 2;
var gp = pltTab.ToGnuplot();
gp.PlotSVG()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m3 O h01 e ta04 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
 
 
 BoSSS-J686 
 
 
 BoSSS-J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,117.1 L519.8,117.1 M71.9,542.4 L73.6,542.4 L75.3,150.3 L77.0,542.4 L78.7,542.4 L80.4,542.2
 L82.1,542.3 L83.8,540.8 L85.5,542.3 L87.2,541.4 L88.9,542.3 L90.6,542.3 L92.2,542.4 L93.9,542.4
 L95.6,542.4 L97.3,542.3 L99.0,131.8 L100.7,129.9 L102.4,542.2 L104.1,542.2 L105.8,126.0 L107.5,124.6
 L109.2,542.1 L110.9,121.7 L112.6,120.6 L114.3,119.7 L116.0,118.5 L117.7,542.2 L119.4,542.2 L121.1,542.3
 L122.8,116.3 L124.5,115.3 L126.2,114.4 L127.9,113.4 L129.5,112.4 L131.2,111.4 L132.9,110.5 L134.6,542.4
 L136.3,542.4 L138.0,542.4 L139.7,106.9 L141.4,105.9 L143.1,540.3 L144.8,541.2 L146.5,102.8 L148.2,103.6
 L149.9,102.0 L151.6,542.4 L153.3,542.4 L155.0,542.4 L156.7,542.4 L158.4,97.4 L160.1,542.4 L161.8,542.4
 L163.5,542.4 L165.2,542.4 L166.8,542.4 L168.5,542.4 L170.2,542.4 L171.9,542.4 L173.6,542.4 L175.3,542.4
 L177.0,93.6 L178.7,93.3 L180.4,93.0 L182.1,92.5 L183.8,92.4 L185.5,92.1 L187.2,91.9 L188.9,91.8
 L190.6,92.0 L192.3,542.4 L194.0,542.4 L195.7,542.4 L197.4,542.4 L199.1,542.4 L200.8,542.3 L202.5,542.3
 L204.1,542.3 L205.8,542.3 L207.5,542.3 L209.2,542.3 L210.9,542.3 L212.6,93.7 L214.3,542.3 L216.0,542.3
 L217.7,542.3 L219.4,542.3 L221.1,542.4 L222.8,542.3 L224.5,542.3 L226.2,98.4 L227.9,542.4 L229.6,542.4
 L231.3,542.4 L233.0,542.4 L234.7,542.4 L236.4,542.4 L238.1,542.4 L239.8,542.4 L241.5,542.4 L243.1,542.4
 L244.8,542.4 L246.5,542.4 L248.2,542.4 L249.9,542.4 L251.6,542.4 L253.3,542.4 L255.0,542.4 L256.7,542.4
 L258.4,542.4 L260.1,542.4 L261.8,542.4 L263.5,542.4 L265.2,542.4 L266.9,542.4 L268.6,542.4 L270.3,542.4
 L272.0,542.4 L273.7,542.4 L275.4,542.4 L277.1,542.4 L278.8,542.4 L280.4,542.4 L282.1,542.4 L283.8,542.4
 L285.5,542.4 L287.2,542.4 L288.9,542.4 L290.6,542.4 L292.3,542.4 L294.0,542.4 L295.7,542.4 L297.4,542.4
 L299.1,542.4 L300.8,542.4 L302.5,542.4 L304.2,542.4 L305.9,542.4 L307.6,542.4 L309.3,140.1 L311.0,542.4
 L312.7,542.4 L314.4,542.4 L316.1,542.4 L317.7,542.4 L319.4,542.4 L321.1,114.9 L322.8,542.4 L324.5,114.4
 L326.2,114.3 L327.9,114.1 L329.6,113.9 L331.3,113.8 L333.0,542.4 L334.7,542.4 L336.4,542.4 L338.1,542.4
 L339.8,542.3 L341.5,542.3 L343.2,542.3 L344.9,542.3 L346.6,542.3 L348.3,542.3 L350.0,542.3 L351.7,542.3
 L353.4,542.2 L355.0,542.2 L356.7,112.2 L358.4,542.2 L360.1,112.2 L361.8,112.3 L363.5,112.3 L365.2,112.3
 L366.9,112.8 L368.6,112.8 L370.3,112.9 L372.0,542.2 L373.7,542.1 L375.4,542.2 L377.1,542.2 L378.8,114.9
 L380.5,115.1 L382.2,115.3 L383.9,115.5 L385.6,115.7 L387.3,116.0 L389.0,116.3 L390.7,116.6 L392.3,116.8
 L394.0,117.0 L395.7,542.4 L397.4,542.4 L399.1,542.4 L400.8,542.4 L402.5,542.4 L404.2,542.4 L405.9,542.4
 L407.6,119.0 L409.3,119.2 L411.0,121.3 L412.7,121.5 L414.4,122.8 L416.1,124.2 L417.8,125.1 L419.5,125.9
 L421.2,126.2 L422.9,126.5 L424.6,126.7 L426.3,127.1 L428.0,127.1 L429.7,127.5 L431.3,127.7 L433.0,542.4
 L434.7,128.2 L436.4,128.6 L438.1,129.1 L439.8,129.5 L441.5,129.9 L443.2,542.3 L444.9,542.3 L446.6,542.3
 L448.3,542.2 L450.0,542.2 L451.7,542.2 L453.4,133.1 L455.1,133.6 L456.8,133.9 L458.5,542.1 L460.2,542.1
 L461.9,542.1 L463.6,542.1 L465.3,136.7 L467.0,137.3 L468.6,137.8 L470.3,138.3 L472.0,139.0 L473.7,139.2
 L475.4,139.8 L477.1,140.2 L478.8,140.7 L480.5,141.2 L482.2,141.7 L483.9,142.1 L485.6,142.6 L487.3,143.0
 L489.0,143.5 L4

In [ ]:
var sess = target.Sessions.Pick(0);
sess

OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04*	04/08/2022 11:34:00	e3a04ed7...

In [ ]:
var ts = sess.Timesteps.Pick(102);
ts

 { Time-step: 100; Physical time: 0.5000000000000003s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }

In [ ]:
SinglePhaseField phi = (SinglePhaseField)ts.GetField("Phi");
phi

Phi

In [ ]:
GridData grdDat = (GridData)phi.GridDat; 
LevelSet lvlSet = new LevelSet(phi.Basis, "levelSet");
lvlSet.Acc(1.0, phi); 
LevelSetTracker lsTrk = new LevelSetTracker(grdDat, XQuadFactoryHelper.MomentFittingVariants.Saye,  1, new string[] {"A", "B"}, lvlSet);
lsTrk.UpdateTracker(0.0);

In [ ]:
using BoSSS.Solution.XNSECommon;

In [ ]:
double[] getMajorAxis3D(LevelSetTracker levSetTrk, LevelSet levSet) {
    
    bool AxisSymmetric = true;

    MultidimensionalArray InterfacePoints = XNSEUtils.GetInterfacePoints(levSetTrk, levSet);

    // droplet width W along x-Axis
    double r_theta90x1 = 0.0;
    double dist_xAxis_min1 = double.MaxValue;
    double r_theta90x2 = 0.0;
    double dist_xAxis_min2 = double.MaxValue;
    // droplet width W along y-Axis
    double r_theta90y1 = 0.0;
    double dist_yAxis_min1 = double.MaxValue;
    double r_theta90y2 = 0.0;
    double dist_yAxis_min2 = double.MaxValue;
    // droplet length L along z-Axis 
    double r_theta0 = 0.0;
    double dist_zAxis_min1 = double.MaxValue;
    double r_theta180 = 0.0;
    double dist_zAxis_min2 = double.MaxValue;

    for(int i = 0; i < InterfacePoints.Lengths[0]; i++) {
        double xCoord = InterfacePoints[i, 0];
        double yCoord = InterfacePoints[i, 1];
        double zCoord = InterfacePoints[i, 2];

        // droplet width W along x-Axis
        double dist_xAxis = Math.Sqrt(yCoord.Pow2() + zCoord.Pow2());
        if(dist_xAxis < dist_xAxis_min1 && xCoord > 0.0) {
            dist_xAxis_min1 = dist_xAxis;
            r_theta90x1 = xCoord;
        }
        if(!AxisSymmetric && dist_xAxis < dist_xAxis_min2 && xCoord < 0.0) {
            dist_xAxis_min2 = dist_xAxis;
            r_theta90x2 = xCoord;
        }


        // droplet width W along y-Axis
        double dist_yAxis = Math.Sqrt(xCoord.Pow2() + zCoord.Pow2());
        if(dist_yAxis < dist_yAxis_min1 && yCoord > 0.0) {
            dist_yAxis_min1 = dist_yAxis;
            r_theta90y1 = yCoord;
        }
        if(!AxisSymmetric && dist_yAxis < dist_yAxis_min2 && yCoord < 0.0) {
            dist_yAxis_min2 = dist_yAxis;
            r_theta90y2 = yCoord;
        }

        // droplet length L along z-Axis 
        double dist_zAxis = Math.Sqrt(xCoord.Pow2() + yCoord.Pow2());
        if(dist_zAxis < dist_zAxis_min1 && zCoord > 0.0) {
            dist_zAxis_min1 = dist_zAxis;
            r_theta0 = zCoord;
        }
        if(dist_zAxis < dist_zAxis_min2 && zCoord < 0.0) {
            dist_zAxis_min2 = dist_zAxis;
            r_theta180 = zCoord;
        }  
        
    }

    double Wx = r_theta90x1 - r_theta90x2;
    if(Wx < 0)
        throw new ArgumentOutOfRangeException("Droplet length Wx is negative");  

    double Wy = r_theta90y1 - r_theta90y2;
    if(Wy < 0)
        throw new ArgumentOutOfRangeException("Droplet length Wy is negative");

    double L = r_theta0 - r_theta180;
    if(L < 0)
        throw new ArgumentOutOfRangeException("Droplet length L is negative");

    return new double[] { r_theta0, L, Wx, Wy };
}

In [ ]:
// Console.WriteLine("Wx = {0}", Wx);  
// Console.WriteLine("Wy = {0}", Wy); 
// Console.WriteLine("L = {0}: r_theta0 = {1}, r_theta180 = {2}", L, r_theta0, r_theta180); 

In [ ]:
var timesteps = sess.Timesteps.Skip(2);
int numTs = timesteps.Count();
numTs

379

In [ ]:
double[] time = new double[numTs];
double[] aspectRatio = new double[numTs];

In [ ]:
int i = 0;
foreach(var tStep in timesteps) {
// for(int tSi = 0; tSi < 10; tSi++) {
//     var tStep = timesteps.Pick(tSi);
    SinglePhaseField phi = (SinglePhaseField)tStep.GetField("Phi");
    GridData grdDat = (GridData)phi.GridDat; 
    LevelSet lvlSet = new LevelSet(phi.Basis, "levelSet");
    lvlSet.Acc(1.0, phi); 
    LevelSetTracker lsTrk = new LevelSetTracker(grdDat, XQuadFactoryHelper.MomentFittingVariants.Saye,  1, new string[] {"A", "B"}, lvlSet);
    lsTrk.UpdateTracker(0.0);

    double[] res = getMajorAxis3D(lsTrk, lvlSet);
    
    time[i] = tStep.PhysicalTime;
    aspectRatio[i] = 0.5 * ((res[1] / (2.0*res[2])) + (res[1] / (2.0*res[3])));
    i++;
}

Error: System.OperationCanceledException: Command :SubmitCode: int i = 0;
foreach(var tStep in timesteps) {
//  ... cancelled.

In [ ]:
Plot2Ddata pltDat = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Blue;
fmt.DashType = DashTypes.Solid;
pltDat.AddDataGroup(sess.ProjectName, time.GetSubVector(0,38), aspectRatio, fmt);
var gp = pltDat.ToGnuplot();
gp.PlotSVG()

In [ ]:
var tStep = timesteps.Pick(130);
tStep

 { Time-step: 130; Physical time: 0.6500000000000005s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }

In [ ]:
phi = (SinglePhaseField)tStep.GetField("Phi");
grdDat = (GridData)phi.GridDat; 
lvlSet = new LevelSet(phi.Basis, "levelSet");
lvlSet.Acc(1.0, phi); 
lsTrk = new LevelSetTracker(grdDat, XQuadFactoryHelper.MomentFittingVariants.Saye,  1, new string[] {"A", "B"}, lvlSet);
lsTrk.UpdateTracker(0.0);

double[] res = getMajorAxis3D(lsTrk, lvlSet);

In [ ]:
res

index,value
0,0.8690092239699065
1,1.973004590158019
2,0.9611486008957587
3,0.9611459872151268


In [ ]:
res

index,value
0,0.8588900064493645
1,2.0833555755569306
2,0.9560944518209614
3,0.9560961945148987
